In [23]:
# Step 1: Training a Model for BSL Alphabets with GPU and Data Augmentation

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers


In [24]:
tf.keras.mixed_precision.set_global_policy('mixed_float16')

In [14]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [25]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
# Setup data directories
data_dir = "./BSL/BSL"  # Replace with the path to your BSL dataset

In [27]:
# Prepare ImageDataGenerator for training and validation data with specific data augmentation
datagen_train = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    zoom_range=0.2,
    fill_mode='nearest',
    validation_split=0.2
)


In [28]:
train_data = datagen_train.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

Found 4020 images belonging to 24 classes.


In [29]:
val_data = datagen_train.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 993 images belonging to 24 classes.


In [30]:
# Model architecture (deeper with more CNN and pooling layers)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_data.num_classes, activation='softmax')
])

In [31]:
num_classes = train_data.num_classes

In [32]:
# Modified model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    
    Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),
    
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # num_classes should be defined
])

In [33]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5, 
    restore_best_weights=True,
    verbose=1  # Optional for more detailed logs
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=3, 
    verbose=1,  # Optional for more detailed logs
    min_lr=1e-6  # Ensures the learning rate does not go too low
)

In [34]:
# Compile the model to use GPU if available
with tf.device('/device:GPU:0'):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Training
    model.fit(train_data, epochs=50, validation_data=val_data, callbacks=[early_stopping, reduce_lr])


Epoch 1/50
126/126 [==============================] - 39s 265ms/step - loss: 5.4515 - accuracy: 0.0552 - val_loss: 9.7359 - val_accuracy: 0.0423 - lr: 0.0010
Epoch 2/50
126/126 [==============================] - 32s 256ms/step - loss: 4.4225 - accuracy: 0.0542 - val_loss: 7.8722 - val_accuracy: 0.0383 - lr: 0.0010
Epoch 3/50
126/126 [==============================] - 32s 256ms/step - loss: 4.2413 - accuracy: 0.0545 - val_loss: 18.6536 - val_accuracy: 0.0403 - lr: 0.0010
Epoch 4/50
126/126 [==============================] - 32s 254ms/step - loss: 4.1033 - accuracy: 0.0552 - val_loss: 14.9754 - val_accuracy: 0.0453 - lr: 0.0010
Epoch 5/50
126/126 [==============================] - 32s 254ms/step - loss: 3.9424 - accuracy: 0.0607 - val_loss: 4.4674 - val_accuracy: 0.0655 - lr: 0.0010
Epoch 6/50
126/126 [==============================] - 32s 252ms/step - loss: 3.8439 - accuracy: 0.0572 - val_loss: 5.2402 - val_accuracy: 0.0483 - lr: 0.0010
Epoch 7/50
126/126 [==============================

In [35]:
# Save model for inference later
model.save('bsl_alphabet1.h5')

In [48]:
from tensorflow.keras import backend as K

# Clear the session to release GPU memory
K.clear_session()
